In [48]:
from pgtools import panaroo_parser
from pgtools.maf_parser import MAFseq, MAF, SyntenyBlock

In [2]:
panaroo_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/panaroo_out/"
gff_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/gff"

In [3]:
panaroo_obj = panaroo_parser.parse_panaroo_output(panaroo_dir, gff_dir, store_seqs=False)

In [10]:
def set_seq(model):
    for seq_coll in model.seq_collections:
        for seq in seq_coll.sequences:
            seq.seq = "-"

set_seq(panaroo_obj)

In [13]:
for seq_coll in panaroo_obj.seq_collections:
    for seq in seq_coll.sequences:
        print(seq.seq)
    break

-
-


In [4]:
edges = panaroo_obj.get_cluster_edges()

In [5]:
clusters_dict = {clust.cluster_name : clust for clust in panaroo_obj.seq_collections}

In [6]:
print(clusters_dict)

{'group_7965': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc64280130>, 'abgT': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc65280160>, 'group_7072': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc64780160>, 'group_265': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc63a00190>, 'group_4537': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc64280220>, 'group_6412': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc63a00250>, 'group_4523': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc65d80280>, 'group_4742': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc65b00280>, 'group_2908': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc64980280>, 'group_2979': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc64280280>, 'group_4180': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc64980340>, 'group_8879': <pgtools.panaroo_parser.PanarooCluster object at 0x7fcc63e80370>, 'group_5623': <pgtools.panaroo_parser.PanarooC

In [7]:
small_edges = edges[:3]

In [41]:
print(small_edges)

[('group_1763', 'group_1916'), ('group_1763', 'rnfG'), ('group_1916', 'tppB')]


In [34]:
src_sizes = {}
for seq_coll in panaroo_obj.seq_collections:
    for seq in seq_coll.sequences:
        if seq.seq_name not in src_sizes:
            src_sizes[seq.seq_name] = seq.src_size

In [71]:
maf_file = "panaroo_complement.maf"
res_maf = open(maf_file, "w")
new_blocks = {}

clusts_dicts = []
for c1, c2 in small_edges:
    clusts = [clusters_dict[c1], clusters_dict[c2]]
    # clust_2 = clusters_dict[c2]
    new_block_seqs = {}
    common_seqs = clusts[0].get_sequence_names().intersection(clusts[1].get_sequence_names())
    # print(common_seqs)
    
    if not common_seqs:
        continue

    # for each sequence: [most expreme coords for one cluster, most extreme poses for the other cluster]
    clusts_seqs_dict = {s:[] for s in common_seqs}
    # print(clusts_seqs_dict)
    for i in range(len(clusts)):
    # c1_seqs = {s.seq_name:None for s in common_seqs}
    # c2_seqs = {s.seq_name:None for s in common_seqs}
        clust_seq = {}
        for s in clusts[i].sequences:
            if s.seq_name not in common_seqs:
                continue
            if s.seq_name in clust_seq:
                s_old = s
                start,end,strand = s_old
                if s_old[0] > s.start:
                    start = s.start
                if s_old[1] < s.end:
                    end = s.end
                clust_seq[s.seq_name] = (start, end, strand)
            else:
                clust_seq[s.seq_name] = (s.start, s.end, s.strand)
        for s, coords in clust_seq.items():
            clusts_seqs_dict[s].append(coords)

    # print(clusts_seqs_dict)
    clusts_dicts.append(clusts_seqs_dict)
    # PROBLEM - CO JEŚLI W JEDNYM KLASTRZE JEDNA SEKWENCJA MA MNIEJSZE POS A NA DRUGIM WIĘKSZE, A INNA SEKWENCJANA ODWÓR?? - RODZIAŁ NA KLASTRY WAŻNNY (NA KOŃCU MOŻNA PO PROSTU PODZIELIĆ SEKWENCJE NA DWIE GRUPY)
print("clusts_dicts",len(clusts_dicts))     
            
    # for seq_name in common_seqs
        # co jeśli jest paralog - wtedy weź sekwencję bliżej tej drugiej
        # if one seq start is bigger than other end - this is the interval for complement
    # in new blocks - left for seqs with lower starts on the left, right for seqs with lower starts on the right

# print(len(clusts_dicts))
# print(clusts_dicts[0])

new_blocks_all = []
for clust in clusts_dicts:
    # print(clust)
    new_blocks = [[],[]]
    for seq, coords in clust.items():
        c1,c2 = coords
        #strands have to be the same
        if c1[-1] != c1[-1]:
            continue
        # print(seq, c1, c2)

        coords.sort(key = lambda x: x[0])
        new_start = coords[0][1]
        new_end = coords[1][0]
        # if sequences from neighbouring blocks are one after another without a gap or are overlapping: skip
        if new_start + 1 >= new_end:
            continue
        # print(coords)
        # print(c1)
        new_seq = MAFseq(seq, new_start, new_end, c1[-1], src_sizes[seq], seq="-")
        if c1[0] > c1[1]:
            new_blocks[1].append(new_seq)
        else:
            new_blocks[0].append(new_seq)

    for B in new_blocks:
        # print(B)
        if len(B) > 1:
            # new_blocks_all.append(SyntenyBlock(B))
            # print(SyntenyBlock(B).to_MAF_block())
            res_maf.write(SyntenyBlock(B).to_MAF_block())
            res_maf.flush()
res_maf.close()

clusts_dicts 3


In [72]:
len(new_blocks_all)

0

In [56]:
print(new_blocks_all)

[<pgtools.maf_parser.SyntenyBlock object at 0x7fcc6b5cea70>, <pgtools.maf_parser.SyntenyBlock object at 0x7fcc6b5cc580>]


In [59]:
for clust in clusts_dicts:
    print("CLUST")
    for seq, coords in clust.items():
        print(seq, coords)

CLUST
5193_2_6.contig00006 [(147322, 148021, 1), (148022, 148658, 1)]
5197_7_4.contig00030 [(1952, 2651, 1), (2652, 3288, 1)]
5150_3_5.contig00001 [(1987, 2686, 1), (2687, 3323, 1)]
5235_5_12.contig00044 [(1785, 2484, -1), (2485, 3121, -1)]
5299_1_3.contig00147 [(2003, 2702, -1), (2703, 3339, -1)]
5235_5_4.contig00291 [(1772, 2471, -1), (2472, 3108, -1)]
5150_2_2.contig00015 [(26407, 27106, 1), (27107, 27743, 1)]
5235_6_6.contig00119 [(1798, 2497, -1), (2498, 3134, -1)]
5235_6_12.contig00241 [(3279, 3978, 1), (3979, 4615, 1)]
5235_6_11.contig00170 [(1790, 2489, -1), (2490, 3126, -1)]
5197_7_5.contig00009 [(184583, 185282, 1), (185283, 185919, 1)]
5299_7_4.contig00190 [(1940, 2639, -1), (2640, 3276, -1)]
5151_2_6.contig00020 [(48659, 49358, 1), (49359, 49995, 1)]
5235_2_1.contig00277 [(1872, 2571, 1), (2572, 3208, 1)]
5193_1_5.contig00101 [(2054, 2753, -1), (2754, 3390, -1)]
5193_8_2.contig00029 [(2003, 2702, -1), (2703, 3339, -1)]
5151_6_6.contig00007 [(172539, 173238, 1), (173239, 173

In [57]:
for block in new_blocks_all:
    print(block.to_MAF_block())

a
s	5150_1_3.contig00001	2062	3	+	520083	-
s	5150_2_2.contig00015	26405	3	+	164363	-
s	5150_3_5.contig00001	1985	3	+	309492	-
s	5151_2_6.contig00020	48657	3	+	83225	-
s	5151_6_6.contig00007	172537	3	+	204574	-
s	5193_1_5.contig00101	2052	3	-	7209	-
s	5193_2_6.contig00006	147320	3	+	187854	-
s	5193_8_2.contig00029	2001	3	-	74017	-
s	5197_2_1.contig00023	1915	3	-	91844	-
s	5197_7_4.contig00030	1950	3	+	62640	-
s	5197_7_5.contig00009	184581	3	+	216796	-
s	5235_1_4.contig00543	385	3	-	3665	-
s	5235_2_1.contig00277	1870	3	+	7025	-
s	5235_5_12.contig00044	1783	3	-	23309	-
s	5235_5_4.contig00291	1770	3	-	6930	-
s	5235_6_11.contig00170	1788	3	-	11335	-
s	5235_6_12.contig00241	3277	3	+	8431	-
s	5235_6_6.contig00119	1796	3	-	12943	-
s	5299_1_3.contig00147	2001	3	-	13155	-
s	5299_7_4.contig00190	1938	3	-	7114	-


a
s	5150_1_3.contig00001	3400	608	+	520083	-
s	5150_2_2.contig00015	27743	602	+	164363	-
s	5150_3_5.contig00001	3323	608	+	309492	-
s	5151_2_6.contig00020	49995	608	+	83225	-
s	5151_6_6.

In [32]:
l = [(5,2),(3,4)]
a,b = l
l.sort()
print(l)
print(a)


[(3, 4), (5, 2)]
(5, 2)


In [18]:
for c1, c2 in small_edges:
    clusts = [clusters_dict[c1], clusters_dict[c2]]
    print(clusts[0].to_MAF_block())
    print(clusts[1].to_MAF_block())

a
s	5150_1_3.contig00001	2064	699	+	520083	-
s	5150_2_2.contig00015	26407	699	+	164363	-
s	5150_3_5.contig00001	1987	699	+	309492	-
s	5151_2_6.contig00020	48659	699	+	83225	-
s	5151_6_6.contig00007	172539	699	+	204574	-
s	5193_1_5.contig00101	2054	699	-	7209	-
s	5193_2_6.contig00006	147322	699	+	187854	-
s	5193_8_2.contig00029	2003	699	-	74017	-
s	5197_2_1.contig00023	1917	699	-	91844	-
s	5197_7_4.contig00030	1952	699	+	62640	-
s	5197_7_5.contig00009	184583	699	+	216796	-
s	5235_1_4.contig00543	387	699	-	3665	-
s	5235_2_1.contig00277	1872	699	+	7025	-
s	5235_5_12.contig00044	1785	699	-	23309	-
s	5235_5_4.contig00291	1772	699	-	6930	-
s	5235_6_11.contig00170	1790	699	-	11335	-
s	5235_6_12.contig00241	3279	699	+	8431	-
s	5235_6_6.contig00119	1798	699	-	12943	-
s	5299_1_3.contig00147	2003	699	-	13155	-
s	5299_7_4.contig00190	1940	699	-	7114	-


a
s	5150_1_3.contig00001	2764	636	+	520083	-
s	5150_2_2.contig00015	27107	636	+	164363	-
s	5150_3_5.contig00001	2687	636	+	309492	-
s	5151_2_6.con

In [17]:
small_edges

[('group_1763', 'group_1916'), ('group_1763', 'rnfG'), ('group_1916', 'tppB')]

In [ ]:
t= (1,2)
t[0] = 3
t